In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

# Community Reinvestment Act Disclosure Files, 2010 to 2018
`ffiec.gov-cra_disclosure_smb-2.1.1` Last Update: 2020-08-11T21:37:46

_Multi-year CRA disclosures for small business originations._


## Documentation Links

* [Disclosure File Specifications](https://www.ffiec.gov/cra/pdf/17FlatDiscSpecs.pdf) File specifications for the 2017 Disclosure Flat File
* [CRA Data Products](https://www.ffiec.gov/cra/craproducts.htm) 
* [Aggregate & Disclosure Flat Files](https://www.ffiec.gov/cra/craflatfiles.htm) Links to download the files.

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@sandiegodata.org), [San Diego Regional Data Library](http://sandiegodata.org)

## Resources

* ** [sb_loan_orig](data/sb_loan_orig.csv)**. Table D1-1, small business disclosure records, for years 2010 to 2015 inclusive

## References
<ul><li> <strong><a href="https://www.ffiec.gov/cra/xls/18exp_discl.zip">discl_18</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/17exp_discl.zip">discl_17</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/16exp_discl.zip">discl_16</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/15exp_discl.zip">discl_15</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/14exp_discl.zip">discl_14</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/13exp_discl.zip">discl_13</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/12exp_discl.zip">discl_12</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/11exp_discl.zip">discl_11</a></strong>. </li><li> <strong><a href="https://www.ffiec.gov/cra/xls/10exp_discl.zip">discl_10</a></strong>. </li><ul>

In [3]:
pkg.resource('sb_loan_orig')

Header,Type,Description
table_id,text,Value is D1-1
respondent_id,text,Assigned by regulatory agency (same as HMDAID if applicable); Right justified with leading zeros
agency,integer,"Values are 1=OCC, 2=FRS, 3=FDIC, or 4=OTS"
year,integer,Four digit year (e.g. 2012)
loan_type,integer,Value is 4 (Small Business)
action,integer,Value is 1 (Originations)
state,integer,FIPS code with leading zeros or blank for totals across all states
county,integer,FIPS code with leading zeros or blank for totals across all counties
msa,integer,"As defined by OMB; Right justified with leading zeros, NA left justified for areas outside of MSA/MD or blank for totals across all MSA/MDs"
assessment_area,text,"Values are 0001 through 9999; Right justified with leading zeros, NA left justified for areas outside of an Assessment Area (including predominately military areas) OR blank for totals across all Assessment Areas"


In [4]:
df = pkg.resource('sb_loan_orig').dataframe()
df.head()

,table_id,respondent_id,agency,year,loan_type,action,state,county,msa,assessment_area,...,num_orig_bus_lt100k,tot_orig_bus_lt100k,num_orig_bus_lt250k,tot_orig_bus_gt100k_lt250k,num_orig_bus_gt250k_lt1m,tot_orig_bus_gt250k_lt1m,num_orig_bus_lt1m,tot_orig_bus_lt1m,num_orig_bus_al,tot_orig_bus_al
0,D1-1,0000000001,1,2018,4,1,1,1,33860,0028,...,0,0,0,0,0,0,0,0,0,0
1,D1-1,0000000001,1,2018,4,1,1,1,33860,0028,...,4,153,0,0,0,0,2,24,0,0
2,D1-1,0000000001,1,2018,4,1,1,1,33860,0028,...,8,254,0,0,0,0,4,19,0,0
3,D1-1,0000000001,1,2018,4,1,1,1,33860,0028,...,24,802,0,0,0,0,18,533,0,0
4,D1-1,0000000001,1,2018,4,1,1,1,33860,0028,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df[df.year==2015].county.nunique()

326

In [6]:
from geoid.acs import County

def mkgeoid(r):
    try:
        return str(County(r.state, r.county))
    except ValueError:
        return None

df['geoid'] = df.apply(lambda r: mkgeoid(r), axis=1)

In [7]:
df.groupby(['geoid','year']).table_id.count().unstack().sort_values(2010, ascending=False).head()

year,2010,2011,2012,2013,2014,2015,2016,2017,2018
geoid,,,,,,,,,
05000US17031,3200.0,3184.0,3216.0,3216.0,3407.0,3325.0,3324.0,3056.0,3248.0
05000US25017,2926.0,2782.0,2624.0,2592.0,2718.0,3007.0,2783.0,2959.0,2846.0
05000US06013,2918.0,3168.0,1296.0,1471.0,1470.0,1534.0,1598.0,1616.0,1584.0
05000US06001,2914.0,2847.0,1615.0,1530.0,1896.0,1811.0,1936.0,1824.0,1872.0
05000US06037,2731.0,2688.0,2768.0,2704.0,2800.0,3024.0,2976.0,3264.0,3424.0


In [8]:
g = df.groupby(['respondent_id','year'])

In [9]:
len(g.groups.keys())

6564

In [10]:
df.split_county.value_counts()

N    5943357
Name: split_county, dtype: int64

In [11]:
num_col = [c for c in df.columns if c.startswith('num')]
num_col

['num_orig_bus_lt100k',
 'num_orig_bus_lt250k',
 'num_orig_bus_gt250k_lt1m',
 'num_orig_bus_lt1m',
 'num_orig_bus_al']

In [12]:
tot_col = [c for c in df.columns if c.startswith('tot')]
tot_col

['tot_orig_bus_lt100k',
 'tot_orig_bus_gt100k_lt250k',
 'tot_orig_bus_gt250k_lt1m',
 'tot_orig_bus_lt1m',
 'tot_orig_bus_al']

In [13]:
cols = [c for c in df.columns if c.startswith('tot_') or c.startswith('num_')]

t = df.groupby(['year','geoid'])[cols].sum()

In [15]:
len(t)

29039